In [ ]:
# Necessary until transformers packages is updated in the Kaggle notebook environment. s
!pip install --upgrade transformers

In [ ]:
import transformers
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

model_id = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

In [ ]:
text = "Министерството на туризма в България е отворило нов аспект към туристическата идея. В основата на техния малък градче, Копривщица, бележим нови промени, които стават далеч от бетона и самота. Администрацията на комуна Копривщица води туристическа политика, която подчертава ролята на земята и културната традиция, нежелая само да развива туризма във вилата Копривщицка. Основната идея е да се създаде механизъм за управление на туристическите потенциали на комуната, за да се повдигнат и качеството и достъпността на туристическите атракции.\n\nМинистерството разпространява и изявява думите на местните жители за предоставянето на услуги, свързани с туризма, а именно развиване на туризма в градчето, чрез инфраструктура и пространството, необходимо за важните исторически и културни атракции.\n\nКопривщицкото министерство създава механизъми за развитие и завладяване на туристическата индустрия, ни предоставяйки право на живот на туристическите агентства и услуги от комуната Копривщица. То предоставя изключително деляне на пари за поддържане на традиционалните храм свети Йоан Пресвети (Зорници) и други стари постройки, асоциирани с мъртвата рилска Коларция, който вид трябва да бъде пазен и достъпен за всеки желаещ.\n\nЗа да процлене най-добро до жителите и да насърчи туристическото участие и развитие на туристическата индустрия, водената политика от Министерството на Копривщица акцентира емблемично на културата и историята. То пробужда емоциите и привлича посетителите с малки детайли и особено с историята на точността. В смисъла, това е противоположено на сплотените общества, които развиват мащабни дела на бетон, за да създадат парадокс и уникалност в контекста на туризма в Копривщица.\n\nПреминавайки от различните български населени места и минавайки през историята на тяхната култура, всеки може да покаже интерестното обичайно място през туристическите очила. Министерството на Туризма в Копривщица не търси само развитие на туризма от бетона, но също така и без да премахне духа на историята. Разработването на туристическите художествени многоуровнени пространства е важният ключов за довършителното туристическо място Копривщица, заявява истината за това редица от странни истории, където българите са пътували и запазвали уникалната традиция. Идейно това е мисълта и историята, която трябва да се пренесе през туризма в България."

messages = [
    {"role": "system", "content": "You are an Expert in machine generated text detection. You can analyze text in various languages and can detect them if they are machine generated or human written."},
    {"role": "user", "content": f"Please response with either machine generated or human written. After analyzing the following text {text}. Answer:"},
]

outputs = pipeline(
    messages,
    max_new_tokens=128,
)
print(outputs[0]["generated_text"][-1])

In [ ]:
import pandas as pd
sub_df = pd.read_json('/kaggle/input/coling-25-task-1/en_devtest_text_id_only.jsonl', lines=True)
sub_df

In [ ]:
# Function to classify the text and return 0 for human, 1 for machine
def classify_text(text):
    # Concatenate system and user prompts into a single string
    prompt = (
        "You are an expert in machine-generated text detection. "
        "and can detect whether it is machine-generated or human-written.\n\n"
        f"Please respond with either 'machine generated' or 'human written'. After analyzing the following text:\n\n{text}\n\nAnswer:"
    )
    
    # Run the pipeline on the prompt
    outputs = pipeline(prompt, max_new_tokens=50)
    
    # Extract the generated prediction
    generated_text = outputs[0]["generated_text"].strip().lower()
#     print("generated_text ", generated_text)

    # Extract the text after "answer:"
    if "answer:" in generated_text:
        answer_text = generated_text.split("answer:")[1].strip()
         # Split the answer_text into words and get the first 5
        words = answer_text.split()[:5]
        prediction = " ".join(words)  # Join the first 5 words back into a string

    else:
        prediction = generated_text

#     print(f"Prediction: {prediction}")
    
    # Map to binary label: 0 for human, 1 for machine
    if "human" in prediction:
        return 0
    else:
        return 1


In [ ]:
from tqdm import tqdm

# Initialize a new column for predictions
sub_df['label'] = 1

predictions = []
# Use tqdm for progress tracking
for i in tqdm(range(len(sub_df)), desc="Classifying text"):
    # Get the text from each row
    text = sub_df.loc[i, 'text']
#     print(text)
    # Classify and assign the result to the 'prediction' column
    final_output = 1

    final_output = classify_text(text)
#     print('final output printing', final_output)
    
    predictions.append(final_output)
    sub_df.loc[i, 'label'] = final_output

# Print the updated DataFrame with predictions
sub_df

In [ ]:
prediction_file_taska = 'task_a_llm.jsonl' 
predictions_df = pd.DataFrame({'id': sub_df.id, 'label': predictions})
predictions_df.to_json(prediction_file_taska, lines=True, orient='records')
predictions_df

In [ ]:
# sub_df['label'] = predictions
sub_df